In [1]:
import pandas as pd
import numpy as np
import cv2
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt

In [2]:
import os

In [3]:
subjects = ['','Kunal kashyap','Elvis Presley']

In [4]:
def detect_face(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #loding face cascade classifier model
    face_cascade = cv2.CascadeClassifier('opencv-files/lbpcascade_frontalface.xml')
    faces = face_cascade.detectMultiScale(gray,scaleFactor = 1.3 , minNeighbors = 5);
    
    if (len(faces) == 0):
        return None,None
    
    (x,y,w,h)= faces[0]
    return gray[y : y + w , x : x + h],faces[0]

In [5]:
def prepare_training_data(data_folder_path):
    dirs = os.listdir(data_folder_path)
    faces = []
    labels = []
    
    for dir_name in dirs:
        if not dir_name.startswith('s'):
            continue;
        label = int(dir_name.replace("s",""))
        label = int(label)
        subject_dir_path = data_folder_path +"/" + dir_name
        
        subject_images_names = os.listdir(subject_dir_path)
        
        for image_name in subject_images_names:
            if image_name.startswith('.'):
                continue;
            image_path = subject_dir_path + '/'+ image_name
            image = cv2.imread(image_path)
            cv2.imshow('training on image..',image)
            cv2.waitKey(100)
            #calling the  face detection model function \
            face , rect = detect_face(image)
            if face is not None:
                faces.append(face)
                labels.append(label)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    return faces , labels

In [6]:
print('preparing data....')
faces , labels = prepare_training_data("training-data")
print('data prepared.')
print('total faces = ',len(faces))
print('total labels :',len(labels))

preparing data....


error: OpenCV(3.4.1) C:\Miniconda3\conda-bld\opencv-suite_1533128839831\work\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


Now we will use classification to classify which image is whom

In [ ]:
# we can use any of these recognizer
#create our LBPH face recognizer 
#face_recognizer = cv2.face_LBPHFaceRecognizer()

#or use EigenFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face_EigenFaceRecognizer()

#or use FisherFaceRecognizer by replacing above line with 
#face_recognizer = cv2.face_FisherFaceRecognizer()

#but the best and fast is LBPH

In [ ]:

import numpy as np

In [ ]:
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [ ]:
face_recognizer.train(faces,labels = np.array(labels))

In [ ]:
def draw_rectangle(img,rect):
    (x,y,w,h) = rect
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
#funtion to write name text on the image to identify who the person is
#passed x,y cordinates
def draw_text(img,text,x,y):
    cv2.putText(img,text,(x,y),cv2.FONT_ITALIC,1.5,(0,255,0),2)

In [ ]:
#THIS FUNTION RECOGNIZE THE PERSON ON THE IMAGE
def predict(test_img):
    img = test_img.copy()
    face , rect = detect_face(img)
    
    label,confidence = face_recognizer.predict(face)
    label_text = subjects[label]
    draw_rectangle(img,rect)
    draw_text(img,label_text,rect[0],rect[1]-5)
    return img

In [ ]:
print('predicting images...')
test_img1 = cv2.imread('test-data/test_data1.jpg')
test_img2 = cv2.imread('test-data/test_data2.jpeg')

predicted_img1 = predict(test_img1)
predicted_img2 = predict(test_img2)

print('prediction complete')
cv2.imshow(subjects[1],cv2.resize(predicted_img1, (400, 500)))
cv2.imshow(subjects[2],cv2.resize(predicted_img2, (400, 500)))
cv2.imwrite('box_image1.jpg',predicted_img1)
cv2.imwrite('box_image2.jpg',predicted_img2)
cv2.waitkey(0)
cv2.destroyAllWindows()
